In [ ]:
!git clone https://github.com/ai-forever/Kandinsky-2
!pip install './Kandinsky-2'
!pip install git+https://github.com/openai/CLIP.git

In [ ]:
import os
from huggingface_hub import hf_hub_url, cached_download
from copy import deepcopy
from omegaconf.dictconfig import DictConfig
from omegaconf import OmegaConf
from PIL import Image
import numpy as np
import pandas as pd

In [ ]:
def get_kandinsky2_1(
    cache_dir="./kandinsky2",
    use_auth_token=None,
):
    cache_dir = os.path.join(cache_dir, "2_1")
    
    prior_name = "prior_fp16.ckpt"
    config_file_url = hf_hub_url(repo_id="sberbank-ai/Kandinsky_2.1", filename=prior_name)
    cached_download(
        config_file_url,
        cache_dir=cache_dir,
        force_filename=prior_name,
        use_auth_token=use_auth_token,
    )

    config_file_url = hf_hub_url(repo_id="sberbank-ai/Kandinsky_2.1", filename="ViT-L-14_stats.th")
    cached_download(
        config_file_url,
        cache_dir=cache_dir,
        force_filename="ViT-L-14_stats.th",
        use_auth_token=use_auth_token,
    )

In [ ]:
get_kandinsky2_1()

In [ ]:
#create data, load your data
pd.DataFrame([['images/test.jpg', 'test']], columns=['image_name', 'caption']).to_csv('test.csv', index=False)
os.mkdir('images')
Image.new('RGB', (768, 768)).save('images/test.jpg')

In [ ]:
config = OmegaConf.load("Kandinsky-2/train_configs/config_prior.yaml")

In [ ]:
config['params_path'] = 'kandinsky2/2_1/prior_fp16.ckpt'
config['clip_mean_std_path'] = 'kandinsky2/2_1/ViT-L-14_stats.th'
config['data']['train']['df_path'] = 'test.csv'
config['num_epochs'] = 1001
config['save_path'] = 'saves'
if not os.path.exists('saves'):
    os.mkdir('saves')

In [ ]:
with open("train_config.yaml", "w") as f:
    OmegaConf.save(config, f)

In [ ]:
!python Kandinsky-2/train_prior.py --config train_config.yaml